In [1]:
import json
import os
import requests
import torch
from dotenv import load_dotenv
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor
from PIL import Image

from utils import (
    TorchDataset,
    convert_tensor_to_base64,
    convert_to_base64,
    load_local_vision_dataset,
)

load_dotenv()


True

In [11]:
ENDPOINT = "https://customvision-dev-aoai.openai.azure.com"
DEPLOYMENT_NAME = "gpt4o-001"
URL = f"{ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview"

HEADERS = {'api-key': os.getenv("AZURE_OPENAI_API_KEY"),
           'Content-Type': 'application/json'}

In [16]:
# prompt = """You are an expert visual inspector for a manufacturing company that makes glass bottles. You will be shown a top-view image of a glass bottle and your task is to identify if it is defective or not. Think step-by-step - first identify if there is a defect or not. Second, if there is a defect, identify the type of defect. **IF** present, the defect can only be of the following types: 1.broken 2.contamination. Third, explain your reasoning for the defect if present. Finally, identify where the defect is located in the image and provide the relative coordinates (between 0-1) of the bounding box enclosing the defect in the format [x_top, y_top, x_bottom, y_bottom]. Please return your response **strictly** as a valid JSON object with the following format:
# {"is_defective": "<yes or no>",
# "reason": "<describe the defect. leave empty if is_defective is no>",
# "defect_type": "<type of defect. leave empty if is_defective is no>",
# "bounding_box": "[x_top, y_top, x_bottom, y_bottom]. leave empty if is_defective is no"}

# Note: If  there is no defect or if you are unsure, please return "is_defective": "no" and leave the other fields empty.
# """

# prompt

prompt = "Describe the image"

In [23]:
def get_request_body(prompt, base64_image, mime_type):
    request_body = {
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": [
                        {
                            "type": "text",
                            "text": f"{prompt}"
                        },
                    {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:{mime_type};base64," + f"{base64_image}",
                            }
                            }
                ]
            }
        ],
        "max_tokens": 300
    }

    return request_body

In [24]:
sample_img_path = "datasets/raw/mvtec-ad/bottle/test/broken_large/000.png"

# base64_image = convert_tensor_to_base64(sample_img)
base64_image, mime_type = convert_to_base64(sample_img_path)
print(mime_type)

request_body = get_request_body(prompt, base64_image, mime_type)

response = requests.post(URL, headers=HEADERS,
                             json=request_body, timeout=120)


image/png


In [26]:
response.json()

{'error': {'code': 'InternalServerError',
  'message': 'The service is temporarily unable to process your request. Please try again later.',
  'param': None,
  'type': None}}